In [83]:
import os
import glob
import pandas as pd 
import zipfile
import geopandas as gpd
from utils import *
from congestion_metrics import *
import logging
# from simpledbf import Dbf5s

In [84]:
LANDUSE_ASSUMPTIONS = pd.read_csv("/Users/tszchun.chow/Documents/GitHub/akl_touch/inputs/land_use_assumptions/Landuse_Detailed_Table_2026_Slot_16.csv", 
                                  skiprows=10).drop(columns='c ')[:-1]
LANDUSE_ASSUMPTIONS['Zone'] = LANDUSE_ASSUMPTIONS['Zone'].astype(int)

employment_cols = [x for x in LANDUSE_ASSUMPTIONS.columns.tolist() if "E" in x]
LANDUSE_ASSUMPTIONS['E'] = LANDUSE_ASSUMPTIONS[employment_cols].sum(axis=1)
employment_assumptions = LANDUSE_ASSUMPTIONS[['Zone', 'E']]

population_cols = [x for x in LANDUSE_ASSUMPTIONS.columns.tolist() if "PT" in x]
LANDUSE_ASSUMPTIONS['Population'] = LANDUSE_ASSUMPTIONS[population_cols].sum(axis=1)
population_assumptions = LANDUSE_ASSUMPTIONS[['Zone', 'Population']]

population_assumptions['Population'].sum()

np.float64(1760162.3574499835)

In [85]:
sector_map = pd.read_csv('../inputs/sector_map/19_Internal_Sect_ga.ens', sep=':', skiprows=27)
sector_map.columns = ['sector', 'zone']
sector_map['sector'] = sector_map['sector'].str.split("ga").str[-1].astype(int)
sector_map['zone'] = sector_map['zone'].astype(int)
sector_map

,sector,zone
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
591,17,592
592,17,593
593,18,594
594,18,595


In [86]:
def make_ijk(temp_df, metric):

    ijk = pd.melt(temp_df, id_vars='origin', var_name='destination', value_name=metric)
    # ijk['metric'] = metric

    return (ijk)

# zip_path = '../inputs/agg_mx/p21_011_Time_GC_Agg.zip'
zip_path = '../inputs/msm_outputs_mx/P2_1 HBW GC Mx disagg.zip'

pt_ijk = []
counter = 0
scenario_mode_list = []

SCEN_MAP = SCENARIO_P1_MAP.copy()

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()

    csv_list = [x for x in file_list if x.endswith('.csv')]
    # pt_csv_list = [x for x in file_list if ("PT" in x.split("/")[-1]) and (x.endswith('.csv'))]

    for csvf in csv_list:

        slot_no = str(int(csvf.split('/')[-1].split("_")[0][2:4])-1)
        trippurpose = csvf.split('/')[-1].split(" ")[2]
        period = csvf.split('/')[-1].split(" ")[1]
        mode = csvf.split('/')[-1].split(" ")[-2]
        metric = csvf.split('/')[-1].split(" ")[-1].replace(".csv", "")

        if period.upper() in PERIOD_MAP:
            period_no = PERIOD_MAP[period.upper()]
            scenario_no = "26" + str(period_no) + str(slot_no)
            scenario_mode_list.append(f'{scenario_no}_{trippurpose}_{mode}')

            with zip_ref.open(csvf) as f:
                temp_df = pd.read_csv(f, skiprows=10).rename(columns={'p/q/[val]': 'origin'})
                temp_ijk = make_ijk(temp_df, f'{scenario_no}_{trippurpose}_{mode}_{metric}')
                
                if counter == 0:
                    gc_ijk = temp_ijk.copy()

                else:
                    gc_ijk = gc_ijk.merge(temp_ijk, on=['origin', 'destination'], how='outer')

                counter += 1
                
                # time_ijks.append(combined_ijk)

gc_ijk

,origin,destination,26114_HBW_Car_GC,26114_HBW_PT_GC,26116_HBW_Car_GC,26116_HBW_PT_GC,26118_HBW_Car_GC,26118_HBW_PT_GC,26120_HBW_Car_GC,26120_HBW_PT_GC,...,26160_HBW_Car_GC,26160_HBW_PT_GC,26162_HBW_Car_GC,26162_HBW_PT_GC,26170_HBW_Car_GC,26170_HBW_PT_GC,26180_HBW_Car_GC,26180_HBW_PT_GC,26182_HBW_Car_GC,26182_HBW_PT_GC
0,1,1,15.768884,4999.5,15.769327,4999.5,15.769352,4999.5,15.768675,4999.5,...,15.768220,4999.5,15.768016,4999.5,15.768690,4999.5,15.767708,4999.5,15.767596,4999.5
1,1,10,59.280529,9999.0,59.283676,9999.0,59.282444,9999.0,59.296017,9999.0,...,59.309490,9999.0,59.309597,9999.0,59.289684,9999.0,59.300217,9999.0,59.303555,9999.0
2,1,100,151.117691,9999.0,148.751923,9999.0,148.712830,9999.0,150.205185,9999.0,...,148.029175,9999.0,145.690155,9999.0,150.459122,9999.0,145.900726,9999.0,145.946320,9999.0
3,1,101,154.608704,9999.0,152.242706,9999.0,152.203873,9999.0,152.006638,9999.0,...,151.519379,9999.0,149.179932,9999.0,155.762650,9999.0,149.392624,9999.0,149.437134,9999.0
4,1,102,151.840057,9999.0,149.343903,9999.0,151.567902,9999.0,150.839874,9999.0,...,148.746582,9999.0,146.408875,9999.0,150.745789,9999.0,146.608719,9999.0,146.577896,9999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489995,99996,99992,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,...,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0
489996,99996,99993,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,...,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0
489997,99996,99994,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,...,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0
489998,99996,99995,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,...,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0


In [82]:
gc_ijk[gc_ijk['origin']==2]['26116_HBW_PT_GC'].min()

np.float64(36.88389587402344)

In [87]:
gc_ijk['origin'] = gc_ijk['origin'].astype(int)
gc_ijk['destination'] = gc_ijk['destination'].astype(int)

gc_access_ijk = gc_ijk.merge(employment_assumptions.rename(columns={'Zone':'destination'}), on='destination')
gc_access_ijk = gc_access_ijk[(gc_access_ijk['origin']<=595)&(gc_access_ijk['destination']<=595)]
accemp_agg = {}

for scenario_mode in scenario_mode_list:
    if "Car" in scenario_mode:
        gc_access_ijk[f'{scenario_mode}_AccEmp'] = (gc_access_ijk[f'{scenario_mode}_GC'].between(0, 30))*gc_access_ijk[f'E'] 
    elif "PT" in scenario_mode:
        gc_access_ijk[f'{scenario_mode}_AccEmp'] = (gc_access_ijk[f'{scenario_mode}_GC'].between(0, 45))*gc_access_ijk[f'E'] 

    accemp_agg[f'{scenario_mode}_AccEmp'] = 'sum'

gc_oaccess_ijk = gc_access_ijk.groupby('origin').agg(accemp_agg).reset_index(
                    ).merge(population_assumptions.rename(columns={'Zone':'origin'}), on='origin')

gc_oaccess_ijk

,origin,26114_HBW_Car_AccEmp,26114_HBW_PT_AccEmp,26116_HBW_Car_AccEmp,26116_HBW_PT_AccEmp,26118_HBW_Car_AccEmp,26118_HBW_PT_AccEmp,26120_HBW_Car_AccEmp,26120_HBW_PT_AccEmp,26122_HBW_Car_AccEmp,...,26160_HBW_PT_AccEmp,26162_HBW_Car_AccEmp,26162_HBW_PT_AccEmp,26170_HBW_Car_AccEmp,26170_HBW_PT_AccEmp,26180_HBW_Car_AccEmp,26180_HBW_PT_AccEmp,26182_HBW_Car_AccEmp,26182_HBW_PT_AccEmp,Population
0,1,1004.32491,0.00000,1004.32491,0.00000,1004.32491,0.00000,1004.32491,0.00000,1004.32491,...,0.00000,1004.32491,0.00000,1004.32491,0.00000,1004.32491,0.00000,1004.32491,0.00000,3495.99163
1,2,1046.19329,0.00000,1046.19329,0.00000,1046.19329,0.00000,1046.19329,0.00000,1046.19329,...,0.00000,1046.19329,0.00000,1046.19329,0.00000,1046.19329,0.00000,1046.19329,0.00000,1287.95761
2,3,1046.19329,0.00000,1046.19329,0.00000,1046.19329,0.00000,1046.19329,0.00000,1046.19329,...,0.00000,1046.19329,0.00000,1046.19329,0.00000,1046.19329,0.00000,1046.19329,0.00000,995.57212
3,4,1046.19329,0.00000,1046.19329,0.00000,1046.19329,0.00000,1046.19329,0.00000,1046.19329,...,0.00000,1046.19329,0.00000,1046.19329,0.00000,1046.19329,0.00000,1046.19329,0.00000,322.88374
4,5,1229.51329,0.00000,1229.51329,0.00000,1229.51329,0.00000,1229.51329,0.00000,1229.51329,...,0.00000,1229.51329,0.00000,1229.51329,0.00000,1229.51329,0.00000,1229.51329,0.00000,4791.39339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,591,9405.28119,0.00000,9405.28119,0.00000,9405.28119,0.00000,9405.28119,0.00000,9405.28119,...,0.00000,9405.28119,0.00000,9405.28119,0.00000,9405.28119,0.00000,9405.28119,0.00000,10269.99656
591,592,3670.41020,0.00000,3670.41020,0.00000,3670.41020,0.00000,3670.41020,0.00000,3670.41020,...,0.00000,3670.41020,0.00000,3670.41020,0.00000,3670.41020,0.00000,3670.41020,0.00000,3064.06246
592,593,1519.04440,0.00000,1519.04440,0.00000,1519.04440,0.00000,1519.04440,0.00000,1519.04440,...,0.00000,1519.04440,0.00000,1519.04440,0.00000,1519.04440,0.00000,1519.04440,0.00000,5712.27714
593,594,4583.53850,0.00000,4583.53850,0.00000,4583.53850,0.00000,4583.53850,0.00000,4583.53850,...,0.00000,4583.53850,0.00000,4583.53850,0.00000,4583.53850,0.00000,4583.53850,0.00000,6550.44618


In [89]:
sumproduct_cols = []
for scenario_mode in scenario_mode_list:
    gc_oaccess_ijk[f'{scenario_mode}_AccEmp*Pop'] = (gc_oaccess_ijk[f'{scenario_mode}_AccEmp'])*gc_oaccess_ijk['Population'] 
    sumproduct_cols.append(f'{scenario_mode}_AccEmp*Pop')

population_total = gc_oaccess_ijk['Population'].sum().item()

gc_oaccess_sum = pd.DataFrame(gc_oaccess_ijk[sumproduct_cols].sum(), columns=['Value']).reset_index()
gc_oaccess_sum.columns = ['Index', 'Value']
gc_oaccess_sum['Scenario'] = gc_oaccess_sum['Index'].str.split("_").str[0].str[-2:].astype(int)
gc_oaccess_sum['Mode'] = gc_oaccess_sum['Index'].str.split("_").str[2]
gc_oaccess_sum['Value'] = gc_oaccess_sum['Value']/population_total
gc_oaccess_sum

,Index,Value,Scenario,Mode
0,26114_HBW_Car_AccEmp*Pop,116518.291632,14,Car
1,26114_HBW_PT_AccEmp*Pop,5879.591508,14,PT
2,26116_HBW_Car_AccEmp*Pop,117035.873702,16,Car
3,26116_HBW_PT_AccEmp*Pop,6021.806230,16,PT
4,26118_HBW_Car_AccEmp*Pop,117867.708459,18,Car
5,26118_HBW_PT_AccEmp*Pop,6097.259459,18,PT
6,26120_HBW_Car_AccEmp*Pop,118758.581818,20,Car
7,26120_HBW_PT_AccEmp*Pop,6182.468695,20,PT
8,26122_HBW_Car_AccEmp*Pop,118643.082094,22,Car
9,26122_HBW_PT_AccEmp*Pop,6182.468695,22,PT


In [79]:
gc_oaccess_ijk.to_csv('../outputs/time_gc/GC_Job_Accessibility.csv', index=None)

In [98]:
gc_oaccess_ijk
value_cols = [x for x in gc_oaccess_ijk.columns if (('AccEmp' in x)&('Pop' not in x))]
gc_oaccess_csv = pd.melt(gc_oaccess_ijk, id_vars=['origin', 'Population'], value_vars=value_cols)
gc_oaccess_csv['mode'] = gc_oaccess_csv['variable'].str.split("_").str[2]
gc_oaccess_csv['trip_purpose'] = gc_oaccess_csv['variable'].str.split("_").str[1]
gc_oaccess_csv['metrics'] = "AcctoEmp-GC"
gc_oaccess_csv['scenario_no'] = gc_oaccess_csv['variable'].str.split("_").str[0]
gc_oaccess_csv['pop*emp'] = gc_oaccess_csv['Population']*gc_oaccess_csv['value']
gc_oaccess_csv = gc_oaccess_csv.rename(columns={'value': 'n_accessible_jobs'})

gc_oaccess_csv[['origin', 'n_accessible_jobs', 'metrics', 'scenario_no', 'mode', 'trip_purpose', 'Population', 'pop*emp']].to_csv('../outputs/time_gc/job_access.csv', index=None)